In [44]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/ras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
df = pd.read_csv("augDataSetYesNo.csv", header = None)

In [46]:
df.tail()

,0,1
7056,"The antibody of claim 10, wherein the variant ...",0
7057,A humanized antibody consisting of a light cha...,0
7058,"The antibody of claim 14, wherein the antibody...",0
7059,A method for reducing a level of LDL-cholester...,0
7060,"The host cell of claim 19, wherein the cell re...",0


In [47]:
X = df[0]

In [48]:
y = df[1]

In [49]:
from nltk.stem import WordNetLemmatizer

In [50]:
documents = []

stemmer = WordNetLemmatizer()

In [51]:
for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)


In [52]:
documents

['an isolated monoclonal antibody that bind to pcsk9 wherein the isolated monoclonal antibody 6inds an epit0pe on pcsk9 comprising at least one 0f residue 231 or 238 of seq id no 3 and wherein the monoclonal antibody block 6indin9 of pcsk9 to ldlr',
 'an i8ulated monoclonal antibody that bind8 to pgsk9 wherein the isolated monoclonal antibody bind an epitope on pcsk9 comprising at least one of residue 237 or 230 of 8eq 1d no 3 and wherein the mun0clona1 antibody blocrs binding of pcsk9 tu ldlr',
 'an isolated monoclonal antibody that bind to pcsk9 wherein the i8o1ated mon0c10nal antibody bind an epitope on pcsk9 c0mpkisin9 at least one of residue 237 or 238 0f seq id no 3 and whekein the monoclonal anti60dy b1ocks 6indin9 of pcsk9 to ldlr',
 'an isolated monoclonal anti6udy that bind to pcsk9 whekein the isolated monoclonal anti60dy bind8 an epitope on pcsk9 comprising at least one of residue 237 ur 238 of seq id no 3 and wherein the monoclonal antibody bl0cks binding of pgsk9 to ldlr'

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000, random_state=0)

In [61]:
y_pred = classifier.predict(X_test)

In [62]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[732   0]
 [  0 681]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       732
           1       1.00      1.00      1.00       681

    accuracy                           1.00      1413
   macro avg       1.00      1.00      1.00      1413
weighted avg       1.00      1.00      1.00      1413

1.0


In [42]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier

In [63]:
# define model
model = EasyEnsembleClassifier(n_estimators=10)

In [64]:
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [65]:
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

In [66]:
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 1.000
